# Demonstrateur pour la plateforme web-cartographique d'indicateurs  - 
# #2. exemples de visualisations de quelques indicateurs


Ce notebook a pour objectif de proposer un premier aperçu visuel de certains indicateurs à travers des cartes interactives.   
Ces visualisations serviront de base de discussion pour la conception de la plateforme web dédiée.  

Les données sont extraites depuis notre base de données **PostgreSQL/PostGIS**.

🛠️ Les principaux outils utilisés sont :
- **Pandas** : pour le traitement et la manipulation des données à partir de la BDD.
- **Folium** : pour la création de cartes interactives en Python.

📋 Les indicateurs affichés :
- Surface en bio
- Surface en BNI (Bas Niveau Intrant)
- Surface non-traitée
- Couverture du sol par les cultures principales 

Les indicateurs ci-dessus ne sont pas affichés sur l'ensemble des découpages géographiques listés dans le Notebook #1.

Ce notebook est un prototype visant à tester la lisibilité et la pertinence des représentations cartographiques.  

In [ ]:
import psycopg2
import geopandas as gpd
import pandas as pd
import json
# pour la visualisation :
from matplotlib import colormaps
import matplotlib.colors as mcolors
import branca.colormap as cm
import folium
from folium.plugins import FloatImage
from folium.plugins import GroupedLayerControl
from folium.plugins import MiniMap, Draw, DualMap
from branca.element import Element

import matplotlib.pyplot as plt
import io
import base64

## Connexion à la BDD

In [ ]:
def connection_2_bdd(uname, password, host, port, database_name):
    """Connection via psycopg2 à la BDD PostgreSQL"""
    return psycopg2.connect(
        user=uname, password=password, host=host, port=port, database=database_name
    )

In [ ]:
# Lecture des params de connexion dans un json dédié
fileObject = open("../param_connexion_BDD.json", mode="r", encoding="utf-8")
jsonContent = fileObject.read()
param_connexion = json.loads(jsonContent)

conn = connection_2_bdd(
    param_connexion["username"],
    param_connexion["password"],
    param_connexion["host"],
    param_connexion["port"],
    param_connexion["bdd_name"],
)
curs = conn.cursor()

## Affichage du territoire

In [ ]:
# Requête SQL pour récupérer toutes les données
sql = 'SELECT * FROM app_webae_dev.territoire_test_aac_coulonge_st_hyppolyte;'

gdf_aac = gpd.read_postgis(sql, conn, geom_col="geom")

# Conversion de la projection
gdf_aac = gdf_aac.to_crs(epsg=4326)

## Communes et indicateurs du territoire

In [ ]:
# Requête SQL pour récupérer toutes les communes et indicateurs sur le territoire
sql = """
WITH communes_territoires AS
(SELECT DISTINCT 'aac_coulonge_st_hyppolyte' AS territoire, a.id_zone
,b.insee_com,nom_commune,insee_dep,nom_departement,insee_reg,nom_region,siren_epci,nom_epci,b.geom
      FROM app_webae_dev.territoire_test_aac_coulonge_st_hyppolyte a
      JOIN app_webae_dev.v_communes2025_fre_4326 b
      ON st_intersects(st_transform(a.geom,4326),st_transform(b.geom,4326))
)
SELECT DISTINCT a.geom,a.nom_commune, b.*
FROM communes_territoires a
LEFT JOIN app_webae_dev.mv_indicateurs2020_com2025_fr b
ON a.insee_com=b.insee_com;
"""

gdf_communes = gpd.read_postgis(sql, conn, geom_col="geom")
gdf_communes = gdf_communes.to_crs(epsg=4326)

## Carreaux de 10km et indicateurs du territoire

In [ ]:
# Requête SQL pour récupérer toutes les communes et indicateurs sur le territoire
sql = """
WITH carreaux_territoire AS (
    SELECT DISTINCT c.id_carreau, c.geom
    FROM app_webae_dev.ign_carreaux_10km_fr_2154 c
    JOIN app_webae_dev.territoire_test_aac_coulonge_st_hyppolyte t
    ON ST_Intersects(c.geom, t.geom)
)
SELECT c.geom, i.*
FROM carreaux_territoire c
JOIN app_webae_dev.mv_indicateurs2020_carreau10km_fr i
ON c.id_carreau = i.id_carreau;
"""

gdf_carreaux = gpd.read_postgis(sql, conn, geom_col="geom")
gdf_carreaux = gdf_carreaux.to_crs(epsg=4326)

## Départements et indicateurs du territoire

In [ ]:
# Requête SQL pour récupérer toutes les départements et indicateurs sur le territoire de l'aac coulonge st hyppolyte
sql = """
WITH dpt_territoire AS (
	SELECT DISTINCT d.insee_dep, d.geom, d.nom
	FROM ref_administratif_2025.ign_admin_dep_fr_2154 d
	JOIN app_webae_dev.territoire_test_aac_coulonge_st_hyppolyte t
	ON ST_Intersects(d.geom, t.geom)
)
SELECT d.geom, d.nom, i.*
FROM dpt_territoire d
JOIN app_webae_dev.mv_indicateurs2020_dep2025_fr i
ON d.insee_dep = i.insee_dep;
"""

gdf_dpt = gpd.read_postgis(sql, conn, geom_col="geom")
gdf_dpt = gdf_dpt.to_crs(epsg=4326)

## Petites Régions Agricoles (PRA) et indicateurs du territoire 

In [ ]:
# Requête SQL pour récupérer toutes les PRA et indicateurs sur le territoire de l'aac coulonge st hyppolyte
sql = """
WITH pra_territoire AS (
	SELECT DISTINCT d.code_pra, d.geom, d.libelle_pra
	FROM ref_agricole.ofb_petites_regions_agricoles_fr_2154 d
	JOIN app_webae_dev.territoire_test_aac_coulonge_st_hyppolyte t
	ON ST_Intersects(d.geom, t.geom)
)
SELECT p.geom, p.libelle_pra, i.*
FROM pra_territoire p
JOIN app_webae_dev.mv_indicateurs2020_pra_fr i
ON p.code_pra = i.code_pra;
"""

gdf_pra = gpd.read_postgis(sql, conn, geom_col="geom")
gdf_pra = gdf_pra.to_crs(epsg=4326)

## Indicateur simple : % SAU BIO

In [ ]:
# Ajout d'une colonne pour calcluer le pourcentage de SAU bio
gdf_pra["pourcentage_bio"] = (100 * gdf_pra["sau_rpgbio"] / gdf_pra["sau_totale"]).round(2)
gdf_dpt["pourcentage_bio"] = (100 * gdf_dpt["sau_rpgbio"] / gdf_dpt["sau_totale"]).round(2)
gdf_communes["pourcentage_bio"] = (100 * gdf_communes["sau_rpgbio"] / gdf_communes["sau_totale"]).round(2)
gdf_carreaux["pourcentage_bio"] = (100 * gdf_carreaux["sau_rpgbio"] / gdf_carreaux["sau_totale"]).round(2)

# Ajoutd'une colonne pour calculer la moyenne de la SAU bio par unité de surface
gdf_pra["pourcentage_bio_moyen"] = gdf_pra["pourcentage_bio"].mean().round(2)
gdf_dpt["pourcentage_bio_moyen"] = gdf_dpt["pourcentage_bio"].mean().round(2)
gdf_communes["pourcentage_bio_moyen"] = gdf_communes["pourcentage_bio"].mean().round(2)
gdf_carreaux["pourcentage_bio_moyen"] = gdf_carreaux["pourcentage_bio"].mean().round(2)

In [ ]:
# Centrage de la carte
center = gdf_aac.geometry.union_all().centroid
map_center = [center.y, center.x]

# Création de la carte
f = folium.Figure(width=800, height=800)
m = folium.Map(location=map_center, zoom_start=9).add_to(f)

# Choisir la colonne d'indicateur
col = 'pourcentage_bio'
col1 = 'pourcentage_bio_moyen'
col2 = 'sau_totale'

# Créer une échelle de couleurs basée sur les valeurs de l'indicateur
# Pour éviter les valeurs extrêmes, on peut utiliser les percentiles :
min_val = gdf_communes[col].quantile(0.05)  # 5e percentile
max_val = gdf_communes[col].quantile(0.95)  # 95e percentile

# Utiliser une palette matplotlib
mpl_colormap = colormaps.get_cmap('Greens')
norm = mcolors.Normalize(vmin=min_val, vmax=max_val)
# Convertir la palette pour Folium
colormap = cm.LinearColormap(
    colors=[mpl_colormap(i) for i in range(mpl_colormap.N)],
    vmin=min_val, vmax=max_val,
    caption='Pourcentage bio (%)'
)

# Fonction de style qui colore selon la valeur
def style_function(feature):
    value = feature['properties'][col]
    color = colormap(value) if value is not None else '#grey'
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    }

# Ajout des carreaux de 10km avec colorbar associé
folium.GeoJson(
    show=False,
    data=gdf_carreaux.to_json(),
    name="carreaux 10km",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['id_carreau', col, col1, col2], 
                                  aliases=['Carreau', 'Pourcentage SAU bio (%)','Moyenne SAU bio par unité géo (%)', 'SAU totale (ha)'])
).add_to(m)
# Ajout des départements avec colorbar associé
folium.GeoJson(
    show=False,
    data=gdf_dpt.to_json(),
    name="départements",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['nom', col, col1, col2], 
                                  aliases=['Département', 'Pourcentage SAU bio (%)','Moyenne SAU bio par unité géo (%)', 'SAU totale (ha)'])
).add_to(m)
# Ajout des PRA avec colorbar associé
folium.GeoJson(
    show=False,
    data=gdf_pra.to_json(),
    name="PRA",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['libelle_pra', col, col1, col2], 
                                  aliases=['PRA', 'Pourcentage SAU bio (%)','Moyenne SAU bio par unité géo (%)', 'SAU totale (ha)'])
).add_to(m)
# Ajout des communes avec colorbar associé
folium.GeoJson(
    show=False,
    data=gdf_communes.to_json(),
    name="communes",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['nom_commune', col, col1, col2], 
                                  aliases=['Commune', 'Pourcentage SAU bio (%)','Moyenne SAU bio par unité géo (%)', 'SAU totale (ha)'])
).add_to(m)

# Ajout de la couche AAC
folium.GeoJson(
    data=gdf_aac.to_json(),
    name="AAC COULONGE-SAINT HIPPOLYTE",
    style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.2}
).add_to(m)

# Ajout de la colormap
colormap.add_to(m)

# Contrôle des couches
folium.LayerControl().add_to(m)

html = """
<style>
#info-box {
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;  /* caché par défaut */
}
#toggle-btn {
    position: fixed;
    bottom: 20px;
    left: 50px;
    z-index: 10000;
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="info-box">
    <b>Précautions de lecture :</b><br>
    ...
</div>

<button id="toggle-btn" onclick="
    var box = document.getElementById('info-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer les précautions';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher les précautions';
    }
">Afficher les précautions</button>
"""
m.get_root().html.add_child(Element(html))

html2 = """
<style>
#desc-box {
    position: fixed;
    top: 50px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;
}
#desc-btn {
    position: fixed;
    top: 20px;
    left: 50px;
    z-index: 10000;
    background-color: #2196F3;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="desc-box">
    <b>Description de l’indicateur :</b><br>
    Cet indicateur représente la proportion de Surface Agricole Utilisée (SAU) certifiée en agriculture biologique dans chaque unité géographique.<br>
    Il permet d’évaluer le niveau d’engagement territorial en bio, par rapport à la moyenne régionale ou nationale.
</div>

<button id="desc-btn" onclick="
    var box = document.getElementById('desc-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer la description';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher la description';
    }
">Afficher la description</button>
"""
m.get_root().html.add_child(Element(html2))

# Ajout du logo Solagro
url_logo = ("https://osez-agroecologie.org/images/env/logo_solagro_hd.png")
FloatImage(url_logo, bottom=2, right=2, width='150px').add_to(m)


In [ ]:
# Sauvegarde de la carte
m.save("carte_PctageSAUBio.html")

## Indicateur une PopUp avancée (hist+table): Cultures principales

In [ ]:
# Liste des colonnes à utiliser
part_cols = [
    "part_Arboriculture",
    "part_Autre céréale",
    "part_Autre culture",
    "part_Autre légumineuse fourragère",
    "part_Autre oléagineux",
    "part_Avoine",
    "part_Bande tampon / bordure",
    "part_Betterave non fourragère / Bette",
    "part_Blé dur",
    "part_Blé tendre",
    "part_Bois pâturé",
    "part_Chanvre",
    "part_Châtaigneraie entretenue par des porcins ou des petits ru",
    "part_Colza",
    "part_Épeautre",
    "part_Fève",
    "part_Féverole",
    "part_Fleur ou plante ornementale",
    "part_Fourrage annuel",
    "part_Jachère",
    "part_Landes",
    "part_Landes (herbe prédominante)",
    "part_Landes (ligneux prédominants)",
    "part_Légume / maraichage",
    "part_Lentille cultivée (non fourragère)",
    "part_Lin",
    "part_Lupin",
    "part_Luzerne",
    "part_Maïs doux",
    "part_Maïs ensilage",
    "part_Maïs grain",
    "part_Mélange de céréales",
    "part_Mélange de légumineuses",
    "part_Méteil (légumineuses prépondérantes)",
    "part_Millet",
    "part_Orge",
    "part_Pois chiche",
    "part_Pois et haricot frais (alimentation humaine)",
    "part_Pois et haricot secs (alimentation humaine)",
    "part_Pois protéagineux",
    "part_Pomme de terre",
    "part_PPAM",
    "part_Prairie permanente",
    "part_Prairie rotation longue",
    "part_Prairie temporaire diversifiée",
    "part_Prairie temporaire - graminée pure",
    "part_Riz",
    "part_Sainfoin",
    "part_Sarrasin",
    "part_Seigle",
    "part_Soja",
    "part_Sorgho",
    "part_Surface boisée",
    "part_Surface non cultivée",
    "part_Surfaces hautement diversifiées (DOM)",
    "part_Tournesol",
    "part_Trèfle",
    "part_Triticale",
    "part_Vesce",
    "part_Vigne"
]


In [ ]:
# Centrage de la carte
center = gdf_aac.geometry.union_all().centroid
map_center = [center.y, center.x]

# Création de la carte
f = folium.Figure(width=800, height=800)
m = folium.Map(location=map_center, zoom_start=9).add_to(f)

# Dictionnaire culture : couleur
culture_to_color = {
    "Blé tendre": "#23E8d0",
    "Vigne": "#800080",
    "Maïs grain": "#FFD700",
    "Orge": "#FFA500",
    "Tournesol": "#F5DEB3",
    "Prairie permanente": "#025F0F",
    "Prairie temporaire diversifiée": " #ABEBC6",
    "Prairie rotation longue": "#71E823",
    "Jachère": "#717D7E",
}

def ajouter_couche_cultures(gdf, nom_couche, couleur_par_defaut="#D3D3D3"):
    layer = folium.FeatureGroup(name=nom_couche, show=False)
    geojson_data = json.loads(gdf.to_json())

    for feature in geojson_data["features"]:
        props = feature["properties"]

        parts_sau = {col.replace("part_", ""): float(val) for col in part_cols if (val := props.get(col.replace("part_", ""))) not in [None, ""] and float(val) and float(val) > 0}
        top5_ha = sorted(parts_sau.items(), key=lambda x: x[1], reverse=True)[:5]

        parts = {col.replace("part_", ""): val for col in part_cols if (val := props.get(col)) is not None and val > 0}
        top5 = sorted(parts.items(), key=lambda x: x[1], reverse=True)[:5]

        parts_bio = {col.replace("part_", ""): val for col in part_cols if (val := props.get(col.replace("part_", "part_bio_"))) is not None and val > 0}
        top5_bio = sorted(parts_bio.items(), key=lambda x: x[1], reverse=True)[:5]

        if not top5:
            continue

        dominante = top5[0][0].strip()
        if dominante not in culture_to_color:
            dominante = "Blé tendre"  # par défaut
        fill_color = culture_to_color.get(dominante, couleur_par_defaut)

        df_top_sau = pd.DataFrame(top5_ha, columns=["Culture", "SAU (ha)"])
        df_top = pd.DataFrame(top5, columns=["Culture", "Part (%)"])
        df_top_bio = pd.DataFrame(top5_bio, columns=["Culture", "Part bio (%)"])

        df_top["Part (%)"] = (df_top["Part (%)"] * 100).round(1)
        df_top_bio["Part bio (%)"] = (df_top_bio["Part bio (%)"] * 100).round(1)

        df_merged = pd.merge(df_top, df_top_sau, on="Culture", how="left")
        df_merged = pd.merge(df_merged, df_top_bio, on="Culture", how="left")

        df_html = df_merged.to_html(index=False, classes="table table-sm table-striped", border=0)

        sau_totale = props.get("sau_totale", "N/A")

        # Création de l'histogramme
        fig, ax = plt.subplots(figsize=(3, 2.5))
        ax.bar(df_top["Culture"], df_top["Part (%)"], color="#5dade2")
        ax.set_ylabel("Part (%)")
        ax.set_xticks(range(len(df_top["Culture"])))
        ax.set_xticklabels(df_top["Culture"], rotation=45, ha='right', fontsize=8)
        ax.set_ylim(0, 100)
        plt.tight_layout()

        # Sauvegarde en mémoire (base64)
        img_buf = io.BytesIO()
        plt.savefig(img_buf, format='png', bbox_inches='tight', dpi=100)
        plt.close(fig)
        img_buf.seek(0)
        img_base64 = base64.b64encode(img_buf.read()).decode('utf-8')
        img_html = f'<img src="data:image/png;base64,{img_base64}" width="200"><br>'

        # Popup HTML combiné
        popup_html = f"""
        <strong>Top 5 cultures</strong><br>
        Surface Agricole Utile (SAU) : <strong>{sau_totale:.1f} ha</strong><br>
        {img_html}
        {df_html}
        """
        popup = folium.Popup(popup_html, max_width=300)

        feature["properties"]["dominante"] = dominante

        folium.GeoJson(
            feature,
            popup=popup,
            style_function=lambda x: {
                'fillColor': culture_to_color.get(x['properties'].get("dominante", "").strip(), couleur_par_defaut),
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.5
            },
            highlight_function=lambda x: {
                'fillColor': culture_to_color.get(x['properties'].get("dominante", "").strip(), couleur_par_defaut),
                'color': 'black',
                'weight': 2,
                'fillOpacity': 0.7
            }
        ).add_to(layer)
    
    layer.add_to(m)

ajouter_couche_cultures(gdf_carreaux, "Top 5 cultures par carreaux")
ajouter_couche_cultures(gdf_communes, "Top 5 cultures par commune")
ajouter_couche_cultures(gdf_dpt, "Top 5 cultures par département")

# Ajout de la couche AAC
folium.GeoJson(
    data=gdf_aac.to_json(),
    name="AAC COULONGE-SAINT HIPPOLYTE",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.2}
).add_to(m)

# Contrôles
folium.LayerControl().add_to(m)

from branca.element import Element
html = """
<style>
#info-box {
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;  /* caché par défaut */
}
#toggle-btn {
    position: fixed;
    bottom: 20px;
    left: 50px;
    z-index: 10000;
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="info-box">
    <b>Précautions de lecture :</b><br>
    ...
</div>

<button id="toggle-btn" onclick="
    var box = document.getElementById('info-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer les précautions';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher les précautions';
    }
">Afficher les précautions</button>
"""
m.get_root().html.add_child(Element(html))

html2 = """
<style>
#desc-box {
    position: fixed;
    top: 50px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;
}
#desc-btn {
    position: fixed;
    top: 20px;
    left: 50px;
    z-index: 10000;
    background-color: #2196F3;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="desc-box">
    <b>Description de l’indicateur :</b><br>
    Cet indicateur représente ...
</div>

<button id="desc-btn" onclick="
    var box = document.getElementById('desc-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer la description';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher la description';
    }
">Afficher la description</button>
"""
m.get_root().html.add_child(Element(html2))


legend_html = """
<style>
#legend {
    position: fixed;
    bottom: 70px;
    right: 50px;
    width: 220px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
}
.legend-title {
    font-weight: bold;
    margin-bottom: 5px;
}
.legend-item {
    display: flex;
    align-items: center;
    margin-bottom: 5px;
}
.legend-color {
    width: 15px;
    height: 15px;
    margin-right: 8px;
    border: 1px solid #000;
}
</style>
<div id="legend">
    <div class="legend-title">Légende des cultures principales</div>
"""

for culture, color in culture_to_color.items():
    color = color.strip()  # enlever espaces potentiels
    legend_html += f"""
    <div class="legend-item">
        <div class="legend-color" style="background-color:{color};"></div>
        <div>{culture}</div>
    </div>
    """

legend_html += "</div>"

m.get_root().html.add_child(Element(legend_html))

# Ajout du logo Solagro
url_logo = ("https://osez-agroecologie.org/images/env/logo_solagro_hd.png")
FloatImage(url_logo, bottom=2, right=2, width='150px').add_to(m)

In [ ]:
# Sauvegarde de la carte
m.save("carte_CulturesPpales_hist.html")

## Visualisation sous forme de camembert :  % cultures traitées  / non-traitées

La taille de chaque camembert est proportionnelle au résultat de l'indicateur sur la zone considérée.

In [ ]:
# Ajout d'une colonne pour calcluer le pourcentage de SAU bio
gdf_pra["pct_surf_non_traitee"] = (100 * gdf_pra["part_surf_non_traitee"]).round(2)
gdf_dpt["pct_surf_non_traitee"] = (100 * gdf_dpt["part_surf_non_traitee"]).round(2)
gdf_communes["pct_surf_non_traitee"] = (100 * gdf_communes["part_surf_non_traitee"]).round(2)
gdf_carreaux["pct_surf_non_traitee"] = (100 * gdf_carreaux["part_surf_non_traitee"]).round(2)

In [ ]:
# Ajout d'une colonne pour calcluer le pourcentage de SAU bio
gdf_pra["pct_surf_traitee"] = 100 - gdf_pra["pct_surf_non_traitee"]
gdf_dpt["pct_surf_traitee"] = 100 - gdf_dpt["pct_surf_non_traitee"]
gdf_communes["pct_surf_traitee"] = 100 - gdf_communes["pct_surf_non_traitee"]
gdf_carreaux["pct_surf_traitee"] = 100 - gdf_carreaux["pct_surf_non_traitee"]

In [ ]:
# Colonnes à représenter dans les camemberts
cat_cols = ['pct_surf_non_traitee', 'pct_surf_traitee']

# Choisir un facteur d’échelle approprié
def scale_size(gdf, sau_min, sau_max):
    sau_value = gdf['sau_totale']
    min_size, max_size = 0.5, 2.0
    if pd.isna(sau_value):
        return 1.0
    return min_size + (max_size - min_size) * (sau_value - sau_min) / (sau_max - sau_min)

# Fonction pour générer un camembert en base64
def create_pie_chart(values, size=1):
    fig, ax = plt.subplots(figsize=(size, size))
    ax.pie(values, colors=['green', 'red'], wedgeprops={'linewidth': 0})
    plt.axis('off')
    buf = io.BytesIO()
    plt.savefig(buf, format='png', transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    return base64.b64encode(buf.getvalue()).decode('utf-8')

# Fonction générique pour ajouter une couche avec polygones + camemberts
def add_layer_with_pies(map_obj, gdf, name, tooltip_field):
    layer = folium.FeatureGroup(name=name, show=False)
    map_obj.add_child(layer)

    folium.GeoJson(
        data=gdf.to_json(),
        style_function=lambda x: {'fillColor': 'black', 'color': 'black', 'weight': 2, 'fillOpacity': 0.2},
        highlight_function=lambda x: {'fillColor': 'black', 'color': 'black', 'weight': 3, 'fillOpacity': 0.7},
        tooltip=folium.GeoJsonTooltip(fields=['sau_totale', 'pct_surf_traitee'],sticky=True, aliases=['SAU (ha)', 'Part surface traitée (%)']),
    ).add_to(layer)

    for _, row in gdf.iterrows():
        try:
            values = [row[col] for col in cat_cols if pd.notnull(row[col])]
            if not values or sum(values) == 0:
                continue
            size = scale_size(row, gdf['sau_totale'].min(), gdf['sau_totale'].max())
            encoded = create_pie_chart(values, size=size)
            icon_px = int(size * 50) # conversion pouces / pixels
            centroid = row['geom'].centroid
            location = [centroid.y, centroid.x]
            icon = folium.CustomIcon(
                icon_image=f'data:image/png;base64,{encoded}',
                icon_size=(icon_px, icon_px)
            )
            marker = folium.Marker(
                location=location,
                icon=icon,
            )
            layer.add_child(marker)
        except Exception as e:
            print(f"Erreur dans {name} ({row.get(tooltip_field, '')}): {e}")

# Centre de la carte
center = gdf_aac.geometry.union_all().centroid
map_center = [center.y, center.x]

# Carte
f = folium.Figure(width=800, height=800)
m = folium.Map(location=map_center, zoom_start=9, control_scale=True).add_to(f)

# Ajout des couches
add_layer_with_pies(m, gdf_carreaux, "Carreaux 10km + Camemberts", "id_carreau")
add_layer_with_pies(m, gdf_dpt, "Départements + Camemberts", "nom")
add_layer_with_pies(m, gdf_pra, "PRA + Camemberts", "libelle_pra")
add_layer_with_pies(m, gdf_communes, "Communes + Camemberts", "nom_commune")

# AAC
folium.GeoJson(
    data=gdf_aac.to_json(),
    name="AAC COULONGE-SAINT HIPPOLYTE",
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.2},
    highlight_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 3, 'fillOpacity': 0.7},
    tooltip=folium.GeoJsonTooltip(fields=['id_zone'],sticky=True, aliases=['AAC'])
).add_to(m)

# Contrôle des couches
folium.LayerControl().add_to(m)

html = """
<style>
#info-box {
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;  /* caché par défaut */
}
#toggle-btn {
    position: fixed;
    bottom: 20px;
    left: 50px;
    z-index: 10000;
    background-color: #4CAF50;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="info-box">
    <b>Précautions de lecture :</b><br>
    ...
</div>

<button id="toggle-btn" onclick="
    var box = document.getElementById('info-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer les précautions';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher les précautions';
    }
">Afficher les précautions</button>
"""
m.get_root().html.add_child(Element(html))

html2 = """
<style>
#desc-box {
    position: fixed;
    top: 50px;
    left: 50px;
    width: 300px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 12px;
    z-index: 9999;
    display: none;
}
#desc-btn {
    position: fixed;
    top: 20px;
    left: 50px;
    z-index: 10000;
    background-color: #2196F3;
    color: white;
    border: none;
    padding: 5px 10px;
    font-size: 12px;
    border-radius: 5px;
    cursor: pointer;
}
</style>

<div id="desc-box">
    <b>Description de l’indicateur :</b><br>
    Cet indicateur représente la part de surface agricole 
    <span style="color:red; font-weight:bold;">traitée</span> et 
    <span style="color:green; font-weight:bold;">non-traitée</span> 
    dans la SAU par chaque unité géographique.
    La taille du camembert est relatif à la surface de SAU de chaque entité.
    Plus le camembert est important plus la SAU est grande dans cette unité géographique<br>
</div>

<button id="desc-btn" onclick="
    var box = document.getElementById('desc-box');
    if (box.style.display === 'none') {
        box.style.display = 'block';
        this.innerText = 'Masquer la description';
    } else {
        box.style.display = 'none';
        this.innerText = 'Afficher la description';
    }
">Afficher la description</button>
"""
m.get_root().html.add_child(Element(html2))

# Ajout du logo Solagro
url_logo = ("https://osez-agroecologie.org/images/env/logo_solagro_hd.png")
FloatImage(url_logo, bottom=2, right=2, width='150px').add_to(m)

In [ ]:
# Sauvegarde de la carte
m.save("carte_CultureTraitee.html")

## Visualisation de deux indicateurs simultanée : Surface non-traitée et surface en BNI

Utilisation de `folium.plugins.DualMap` pour afficher des cartes qui s'activent simultanément

In [ ]:
# Ajout d'une colonne pour calcluer le pourcentage de surface en BNI
gdf_pra["part_surf_bni"] = (100 * gdf_pra["part_bni"]).round(2)
gdf_dpt["part_surf_bni"] = (100 * gdf_dpt["part_bni"]).round(2)
gdf_communes["part_surf_bni"] = (100 * gdf_communes["part_bni"]).round(2)
gdf_carreaux["part_surf_bni"] = (100 * gdf_carreaux["part_bni"]).round(2)

In [ ]:
def get_colormap(gdf, column, cmap_name='Greens', caption=''):
    """Crée une colormap linéaire basée sur les percentiles."""
    vmin = gdf[column].quantile(0.05).round(1)
    vmax = gdf[column].quantile(0.95).round(1)
    mpl_colormap = colormaps.get_cmap(cmap_name)
    
    return cm.LinearColormap(
        colors=[mpl_colormap(i) for i in range(mpl_colormap.N)],
        vmin=vmin, vmax=vmax,
        caption=caption,
    ), vmin, vmax

def style_function_factory(colormap, col):
    """Crée une fonction de style pour la coloration selon une colonne."""
    def style_function(feature):
        value = feature['properties'].get(col)
        color = colormap(value) if value is not None else 'grey'
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        }
    return style_function

def add_geojson_layer(map_obj, gdf, name, colormap, col, tooltip_fields, tooltip_aliases):
    folium.GeoJson(
        data=gdf.to_json(),
        name=name,
        show=False,
        style_function=style_function_factory(colormap, col),
        highlight_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 3, 'fillOpacity': 0},
        tooltip=folium.GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases)
    ).add_to(map_obj)

# Centrage de la carte
center = gdf_aac.geometry.union_all().centroid
map_center = [center.y, center.x]
m = DualMap(location=map_center, tiles=None, zoom_start=8)

for side in [m.m1, m.m2]:
    folium.TileLayer("OpenStreetMap").add_to(side)
    # Ajout de la couche AAC
    folium.GeoJson(
        data=gdf_aac.to_json(),
        name="AAC COULONGE-SAINT HIPPOLYTE",
        style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.2},
        highlight_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 3, 'fillOpacity': 0.7},
        tooltip=folium.GeoJsonTooltip(fields=['id_zone'], sticky=True, aliases=['AAC'])
    ).add_to(side)

# ---------- Surf non-traitée ----------
col_nontraitee = 'pct_surf_non_traitee'
col_bni = 'part_surf_bni'
colormap_nontraitee, _, _ = get_colormap(gdf_communes, col_nontraitee, cmap_name='viridis', caption='Pourcentage SAU non-traitée (%)')
from branca.element import MacroElement, Template
# Créer un template HTML pour afficher la colormap_nontraitee à droite
legend_html = colormap_nontraitee._repr_html_()
legend_template = Template(f"""
{{% macro html(this, kwargs) %}}
<div style="position: fixed;
            top: 70px; left: 0vw; width: 200px; z-index:9999;
            background-color: transparent; padding: 10px; border:0px ;">
    {legend_html}
</div>
{{% endmacro %}}
""")
legend_element = MacroElement()
legend_element._template = legend_template
# Ajouter ce composant à la carte de droite
m.m1.get_root().add_child(legend_element)

add_geojson_layer(m.m1, gdf_carreaux, "carreaux 10km", colormap_nontraitee, col_nontraitee,
                  ['id_carreau', col_nontraitee, col_bni, 'sau_totale'],
                  ['Carreau', 'Pourcentage SAU non-traitée (%)', 'Pourcentage surface en BNI (%)', 'SAU totale (ha)'])

add_geojson_layer(m.m1, gdf_dpt, "départements", colormap_nontraitee, col_nontraitee,
                  ['nom', col_nontraitee, col_bni, 'sau_totale'],
                  ['Département', 'Pourcentage SAU non-traitée (%)', 'Pourcentage surface en BNI (%)', 'SAU totale (ha)'])

add_geojson_layer(m.m1, gdf_pra, "PRA", colormap_nontraitee, col_nontraitee,
                  ['libelle_pra', col_nontraitee, col_bni, 'sau_totale'],
                  ['PRA', 'Pourcentage SAU non-traitée (%)', 'Pourcentage surface en BNI (%)', 'SAU totale (ha)'])

add_geojson_layer(m.m1, gdf_communes, "communes", colormap_nontraitee, col_nontraitee,
                  ['nom_commune', col_nontraitee, col_bni, 'sau_totale'],
                  ['Commune', 'Pourcentage SAU non-traitée (%)', 'Pourcentage surface en BNI (%)', 'SAU totale (ha)'])

# ---------- SURF en BNI ----------
col_bni = 'part_surf_bni'
colormap_bni, _, _ = get_colormap(gdf_communes, col_bni, cmap_name='plasma', caption='Pourcentage surface en BNI (%)')
from branca.element import MacroElement, Template
# Créer un template HTML pour afficher la colormap_bni à droite
legend_html = colormap_bni._repr_html_()
legend_template = Template(f"""
{{% macro html(this, kwargs) %}}
<div style="position: fixed;
            top: 70px; right: 35vw; width: 200px; z-index:9999;
            background-color: transparent; padding: 10px; border:0px ;">
    {legend_html}
</div>
{{% endmacro %}}
""")
legend_element = MacroElement()
legend_element._template = legend_template
# Ajouter ce composant à la carte de droite
m.m2.get_root().add_child(legend_element)

add_geojson_layer(m.m2, gdf_carreaux, "carreaux 10km", colormap_bni, col_bni,
                  ['id_carreau', col_bni,col_nontraitee, 'sau_totale'],
                  ['Carreau', 'Pourcentage surface en BNI (%)', 'Pourcentage SAU non-traitée (%)', 'SAU totale (ha)'])

add_geojson_layer(m.m2, gdf_dpt, "départements", colormap_bni, col_bni,
                  ['nom', col_bni, col_nontraitee, 'sau_totale'],
                  ['Département', 'Pourcentage surface en BNI (%)', 'Pourcentage SAU non-traitée (%)', 'SAU totale (ha)'])

add_geojson_layer(m.m2, gdf_pra, "PRA", colormap_bni, col_bni,
                  ['libelle_pra', col_bni, col_nontraitee, 'sau_totale'],
                  ['PRA', 'Pourcentage surface en BNI (%)', 'Pourcentage SAU non-traitée (%)', 'SAU totale (ha)'])

add_geojson_layer(m.m2, gdf_communes, "communes", colormap_bni, col_bni,
                  ['nom_commune', col_bni, col_nontraitee, 'sau_totale'],
                  ['Commune', 'Pourcentage surface en BNI (%)', 'Pourcentage SAU non-traitée (%)', 'SAU totale (ha)'])

# Contrôles
folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
# Sauvegarde de la carte
m.save("carte_DualMap_SurfTraitée_SurfBNI.html")